In [ ]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', None)

In [ ]:
df_check_translate = pd.read_csv('Updated_Translated_Campaign_for_Narrative.csv', lineterminator = '\n')

In [ ]:
df_check_translate.loc[df_check_translate.Translated_Tweets == 'Empty Tweet']

In [ ]:
df_alt_influence = pd.read_csv('AltNews Report Check.csv')

In [ ]:
df_alt_report = pd.read_csv('AltNews Report Number.csv')

In [ ]:
df_alt_influence.groupby('Campaign').count()

In [ ]:
df_alt_report_new = pd.merge(df_alt_influence, df_alt_report, on = 'News_Number')

In [ ]:
df_alt_report_new.groupby('Topic').count()

In [ ]:
df_alt_report_new.loc[(df_alt_report_new.Campaign != 'Influence') & 
                      (df_alt_report_new.Topic != 'Politics') & 
                      (df_alt_report_new.Topic != 'Religion')]

In [ ]:
df_alt_report_new = df_alt_report_new.drop(columns = ['Fact_Check_Link_y', 'SL'])

In [ ]:
df_alt_report_new  = df_alt_report_new.rename(columns = {'Fact_Check_Link_x':'Fact_Check_Link'})

In [ ]:
df_alt_report_nonInfluence = df_alt_report_new.loc[(df_alt_report_new.Campaign != 'Influence') & 
                      (df_alt_report_new.Topic != 'Politics') & 
                      (df_alt_report_new.Topic != 'Religion')]

In [ ]:
df_alt_tweet = pd.read_csv('Collected_Fake_Tweets_Still_Available.csv')

In [ ]:
df_alt_tweet.columns

In [ ]:
df_non_influence = pd.merge(df_alt_tweet, df_alt_report_nonInfluence, on = 'Fact_Check_Link')

In [ ]:
df_non_influence

In [ ]:
df_non_influence.groupby('User_ID').count()

In [ ]:
df_cluster_tweet = pd.read_csv('Cluster_Tweet_Final.csv')

In [ ]:
df_cluster_tweet.columns

In [ ]:
df_cluster_tweet = df_cluster_tweet.dropna(subset=['Cluster_ID'])

In [ ]:
df_cluster_tweet = df_cluster_tweet.dropna(subset = 'User_ID')

In [ ]:
df_cluster_tweet.User_ID = df_cluster_tweet.User_ID.astype(int)

In [ ]:
len(set(df_non_influence.User_ID.tolist()))

In [ ]:
df_non_influence_cluster = df_cluster_tweet[df_cluster_tweet.User_ID.isin(df_non_influence.User_ID.tolist())]

In [ ]:
df_non_influence_cluster[['Cluster_ID', 'Tweet_Text']].sample(20)

In [ ]:
df_non_influence_cluster.loc[df_non_influence_cluster.Cluster_ID == 2078].columns

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-hi-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-hi-en")

In [ ]:
from transformers import pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")

In [ ]:
def translator(text):
    input_ids = tokenizer.encode(text, return_tensors="pt", padding=True)
    outputs = model.generate(input_ids)
    decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_text

texts = ["रेलवे निजीकरण हटाओ सरकारी नौकरी बचाओ आरक्षण बचाओ संविधान बचाओ। #SaveRailwaySaveNation",
        "मैं आशा करता हूं कि आगे और भी ज्यादा संख्या में महिलाएं पुलिस सेवा में शामिल होंगी, हमारे देश की ",
         "तिरंगे संग भारत जोड़ने के लिए निकला जनता का हुजूम."]
        

for text in texts:
  print("English Text: ", text)
  print("Hindi Translation: ", translator(text))
  print("*"*50,"\n") 

In [ ]:
for text in texts:
    print(pipe(text))

In [ ]:
texts2= df_cluster_tweet.loc[df_cluster_tweet.Tweet_Lang == 'hi'][['Cluster_ID', 'Tweet_ID', 'Tweet_Text', 'Tweet_Lang']].sample(100).Tweet_Text.tolist()

In [ ]:
need_text = df_cluster_tweet.loc[df_cluster_tweet.Tweet_Lang == 'hi'][['Cluster_ID', 'Tweet_ID', 'Tweet_Text', 'Tweet_Lang']].drop_duplicates('Cluster_ID').Tweet_Text.tolist()

In [ ]:
need_id = df_cluster_tweet.loc[df_cluster_tweet.Tweet_Lang == 'hi'][['Cluster_ID', 'Tweet_ID', 'Tweet_Text', 'Tweet_Lang']].drop_duplicates('Cluster_ID').Cluster_ID.tolist()

In [ ]:
from transformers import pipeline

pipe2 = pipeline("translation", model="facebook/nllb-200-distilled-600M", src_lang='hin_Deva', tgt_lang='eng_Latn',max_length = 800)

In [ ]:
trans_tweet = []
trans_tweet_id = []

In [ ]:
import time

start = time.time()

k = 0
for text in need_text:
    try:
        trans_tweet.append(pipe2(text)[0]['translation_text'])
        
    except:
        trans_tweet.append('Error_Happened')
        
    trans_tweet_id.append(need_id[k])
    k = k + 1
    if k%10000==0:
        with open("translated_tweet_" + str(k) + ".json", "w") as f:
            json.dump(trans_tweet, f)
        with open("translated_tweet_ID" + str(k) + ".json", "w") as f1:
            json.dump(trans_tweet_id, f1)
            
end = time.time()
print(end - start)

In [ ]:
len(trans_tweet)

In [ ]:
with open("translated_tweet_" + 'final' + ".json", "w") as f:
    json.dump(trans_tweet, f)
with open("translated_tweet_ID" + 'final' + ".json", "w") as f1:
    json.dump(trans_tweet_id, f1)